# DS7331 Project 2
#### Group 2: Hollie Gardner, Cleveland Johnson, Shelby Provost
[Dataset Source](https://archive-beta.ics.uci.edu/ml/datasets/census+income)<br/>
[Github Repo](https://github.com/ShelbyP27/DS7331-Project)

In [3]:
#import libraries
import pandas as pd
import numpy as np
import os
import sklearn as sk
import lazypredict

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# data preprocessing 
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.pipeline import Pipeline

#prediction models
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics as mt
from sklearn.neighbors import KNeighborsClassifier

#exceptional work
from lazypredict.Supervised import LazyClassifier, LazyRegressor
from sklearn import datasets


XGBoostError: XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed (vcomp140.dll or libgomp-1.dll for Windows, libomp.dylib for Mac OSX, libgomp.so for Linux and other UNIX-like OSes). Mac OSX users: Run `brew install libomp` to install OpenMP runtime.
  * You are running 32-bit Python on a 64-bit OS
Error message(s): ['dlopen(/opt/anaconda3/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib\n  Referenced from: /opt/anaconda3/lib/python3.8/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: image not found']


## Data Preparation: Part 1
*Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis.*


### Loading and Prepping Data 


In [ ]:
# Importing the census dataset using pandas
# Reading the CSV file after converting file to csv and removing superfluous spaces via Excel.
df = pd.read_csv('https://raw.githubusercontent.com/ShelbyP27/DS7331-Project/main/adult-data.csv')

# Getting a first look at the dataset
df.head()

In [ ]:
#Cleaning up data set
df = df.replace(to_replace='?',value=np.nan) # replace '?' with NaN (not a number)
df.dropna(inplace=True) # Removing na values
df.duplicated(subset=None, keep='first') #Remove duplicates
df['income'] = df['income'].map({'<=50K': 0, '>50K': 1}).astype(int) #One-hot respone

In [ ]:

# One-hot encoding the Categorical variables
if 'sex' in df:
    df['IsMale'] = df.sex == 'Male'
    df.IsMale = df.IsMale.astype(np.int64)
    del df['sex']
    
if 'marital-status' in df:
    tmp_df = pd.get_dummies(df['marital-status'], prefix = 'Marital')
    df = pd.concat((df, tmp_df), axis =1)
    del df['marital-status']
    
if'relationship' in df:
    tmp_df = pd.get_dummies(df['relationship'], prefix = 'Rel')
    df = pd.concat((df, tmp_df), axis =1)
    del df['relationship']

if 'race' in df:
    tmp_df = pd.get_dummies(df['race'], prefix = 'Race')
    df = pd.concat((df, tmp_df), axis =1)
    del df['race']

if 'workclass' in df:
    tmp_df = pd.get_dummies(df['workclass'], prefix = 'Work')
    df = pd.concat((df, tmp_df), axis =1)
    del df['workclass']

if 'occupation' in df:
    tmp_df = pd.get_dummies(df['occupation'], prefix = 'Occupation')
    df = pd.concat((df, tmp_df), axis =1)
    del df['occupation']

if 'education' in df:
    tmp_df = pd.get_dummies(df['education'], prefix = 'Education')
    df = pd.concat((df, tmp_df), axis =1)
    del df['education']

    
#Replace Native Country with Immigrant atribute
if 'native-country' in df:
    df['immigrant'] = np.where(df['native-country']!= 'United-States', 1, 0)
    del df['native-country']
df.head()
    

In [ ]:
# Separate features from the response
if 'income' in df:
    y = df['income'].values
    del df['income']
    X = df.values

# Train / Test split with scaled_X
scaled_X = StandardScaler().fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(scaled_X, y, test_size = .2, random_state=1)

# Train / Test split
#sc = StandardScaler()
#sc.fit(X)
#x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state=1)

# scaledX = sc.fit_transform(X)
# s_x_train, s_x_test, s_y_train, s_y_test = train_test_split(scaledX, y, test_size = .2, random_state=1)

# print(s_x_train)
# print(x_train)

In [ ]:
#standalone PCA for dimension reduction
#https://analyticsindiamag.com/principal-component-analysis-in-python/#:~:text=pca%20%3D%20PCA(n_components%20%3D%20number,explained_variance%20%3D%20pca.explained_variance_ratio_
pca = PCA(n_components = 20)
pca.fit(scaled_X)
variance = pca.explained_variance_ratio_

#scree plot
PC_values = np.arange(pca.n_components_) + 1
plt.plot(PC_values, pca.explained_variance_ratio_, 'o-', linewidth=2, color='blue')
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.show()

In [ ]:
# Pipeline to search for the best combination of PCA truncation and classifier regularization.
# https://scikit-learn.org/stable/auto_examples/compose/plot_digits_pipe.html#sphx-glr-auto-examples-compose-plot-digits-pipe-py

pca = PCA()

# set the tolerance to a large value to make the example faster
logistic = LogisticRegression(max_iter=10000, tol=0.1)
pipe = Pipeline(steps=[("pca", pca), ("logistic", logistic)])

# Set parameters of pipelines
param_grid = {
    "pca__n_components": [1, 5, 15, 30, 45, 60, 65],
    "logistic__C": np.logspace(-4, 4, 4),
}
search = GridSearchCV(pipe, param_grid, n_jobs=2)
search.fit(scaled_X, y)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

# Plot the PCA spectrum
pca.fit(scaled_X)

fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, figsize=(6, 6))
ax0.plot(
    np.arange(1, pca.n_components_ + 1), pca.explained_variance_ratio_, "+", linewidth=2
)
ax0.set_ylabel("PCA explained variance ratio")

ax0.axvline(
    search.best_estimator_.named_steps["pca"].n_components,
    linestyle=":",
    label="n_components chosen",
)
ax0.legend(prop=dict(size=12))

# For each number of components, find the best classifier results
results = pd.DataFrame(search.cv_results_)
components_col = "param_pca__n_components"
best_clfs = results.groupby(components_col).apply(
    lambda g: g.nlargest(1, "mean_test_score")
)

best_clfs.plot(
    x=components_col, y="mean_test_score", yerr="std_test_score", legend=False, ax=ax1
)
ax1.set_ylabel("Classification accuracy (val)")
ax1.set_xlabel("n_components")

plt.xlim(-1, 70)

plt.tight_layout()
plt.show()

## Data Preparation: Part 2
*Describe the final dataset that is used for classification/regression (include a description of newly formed variables you created*

The final dataset that will be used for classification of individuals with an income greater than 50k includes 63 predictor variables. Seven of the eight original categorical variables, including sex, marital status, relationship, race, workclass, occupation, and education, have been one-hot encoded, resulting in a larger number of predictor variables. The eighth original categorical variable, native country, has been trasnformed to a binary variable indicitive of immigrant status; records with native country as 'United States' were assigned a value of 0 while all else was assigned a value of 1. 

## Modeling and Evaluation: Part 1

*Choose and explain your evaluation metrics that you will use (i.e., accuracy, precision, recall, F-measure, or any metric we have discussed). Why are the measures appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions.*

We will be using accuracy as our evaluation metric to compare across multiple types of models. Accuracy is a reasonable metric to use for this comparison since it portrays how well the model performs, and each model can assess for accuracy so the evaluation metric will be the same. Within each model assessment, various evaluation metrics derived from the models will be considered during the refinement process. 

Precision and recall may be considered depending on the needs of the client. But at this time there is not a need to determine a models performance based on the comparison of false negatives or false positives, so accuracy will suffice.  

## Modeling and Evaluation: Part 2

*Choose the method you will use for diving your data into training and testing splits (i.e. are you using Stratified 10-fold cross validation? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate*

In preparation of classification and regression modeling, the final dataset has been given an 80/20 randomized split on the X's and the y to form the tables x_train, x_test, y_train, and y_test. The x_train table will be used to build each model, then the y_train values will be compared to the predicted values of the x_train model's to asses each's accuracy. The use of the x_test and y_test tables are to assess the accuracy of the models' predictions on a subset of data that was not used in the building of the model. This form of splitting is appropriate as our datasize is large enough. 

## Modeling and Evaluation: Part 3

*Create three different classification/regression models (e.g. random forest, KNN, and SVM). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chose metric.*


### Model 1: Logistic Regression

In [ ]:
#Logistic Regression

lr = LogisticRegression(C=1.0, random_state=1, solver='lbfgs')

lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)

print('accuracy', mt.accuracy_score(y_test, y_pred))
print('confusion matrix\n', mt.confusion_matrix(y_test, y_pred))

In [ ]:
weights = lr.coef_.T
variable_names = df.columns
for coef, name in zip(weights, variable_names):
    print (name, "has weight of", coef[0])

In [ ]:
from matplotlib import pyplot as pyplot
%matplotlib inline
plt.style.use('ggplot')

weights = pd.Series(lr.coef_[0], index=df.columns)
weights.plot(kind='bar')
plt.show()



In [ ]:
import plotly
plotly.offline.init_notebook_mode() # run at the start of every notebook

graph1 = {'x': df.columns,
          'y': weights,
       'type': 'bar'}

fig = dict()
fig['data'] = [graph1]
fig['layout'] = {'title': 'Logistic Regression Weights, with error bars'}

plotly.offline.iplot(fig)


In [ ]:
import math
# Age Interpretation 
print(math.exp(-0.006799438255839271))
# Marital_Married-civ-spouse interpretation
print(math.exp(0.00037492480181653277))

### Initial Logistic Regression ### 
<p>The features of importance are notated as those furthest from 0 as this would simulate the greates changes within the model. In this initial logistic regaression model with all attributes included, the features that appear to be of importance are age, hours-per-week, capital loss, education-num, Marital_Never_married, Marital_Married-civ-spouse, Work_Private, Rel_Husband, capital-gain, and Rel_Not-in-family. For example, if the age of an individual increases by one unit, the esimated odds of having an income greater than 50k change by a factor of 0.99. In other words, the odds decrease by 1%. In another example, if a person is a married civilian spouse, the estimated odds of having an income greater than 50k is .03% higher than someone who is not a married civilian spouse. 

In [ ]:
Xnew = df[['age','hours-per-week', 'capital-loss','education-num','Marital_Never-married','Marital_Married-civ-spouse','Work_Private', 'Rel_Husband','capital-gain','Rel_Not-in-family']].values

sc.fit(Xnew)
x_train, x_test, y_train, y_test = train_test_split(Xnew, y, test_size = .2, random_state=1)
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)

print('accuracy', mt.accuracy_score(y_test, y_pred))
print('confusion matrix\n', mt.confusion_matrix(y_test, y_pred))


graph1 = {'x': ['age','hours-per-week', 'capital-loss','education-num','Marital_Never-married','Marital_Married-civ-spouse','Work_Private', 'Rel_Husband','capital-gain','Rel_Not-in-family'],
        'y': weights,
        'type': 'bar'}

fig = dict()
fig['data'] = [graph1]
fig['layout'] = {'title': 'Logistic Regression Weights, with error bars'}

plotly.offline.iplot(fig)


In [ ]:
# Capital Loss interpretation
math.exp(-0.001613973)

#### Simplified Logistic Regression Model ###
<p>In this simplified logistic regression model, the features that appear to be the most important are age, capital-loss, and Marital_Married-civ-spouse. For an example, each one unit increase in capital loss, the odds of having an income over the age of 50k changes by 0.998. In other words, the odds decrease 0.2%.

### Model 2: SVM

In [ ]:
#SVM

train_svm = SVC(kernel = 'rbf', gamma=.1, C=1.0)

train_svm.fit(x_train, y_train)

y_pred = train_svm.predict(x_test)

print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('Classification Error: %.3f' % (1 - (accuracy_score(y_test, y_pred))))

In [ ]:
# Support vecors 
train_svm.support_vectors_

### Non-linear SVM Model ###
The chosen support vectors for this model show the coordinates of the observations that are closest to the hyperplane, or decision boundary. These points influence the position and orientation of the decision boundary to maximize the accuracy of the classifier. 

### Explanation ###
<p>The initial logistic regression model with all attributes included, resulted in a prediction accuracy of 78.6%. While the speed of this model prediction was quick, a second logistic model was ran with the top 10 attributes of importance by weight. The second logistic regression model, including the atributes age, hours-per-week, capital-loss, education-num, Marital_Never-married,Marital_Married-civ-spouse, Work_Private, Rel_Husband, capital-gain, and Rel_Not-in-family, resulted in a predition accuracy of 79.9%. In addition to the prediction accuracy being better in the second logistic regression model, the speed of the model prediction is quicker as well since there are less varibles being included in the model. Finally, the non-linear SVM model, with the same attributes as the second logistic regression model, resulted in a prediction accuracy of 83.6%. The non-linear SVM model resulted in the best prediction accuracy, however the trade-off is in the increased time it takes to run this model as opposed to the logistic regression models. In terms of efficiency in prediction, the non-linear SVM model is better than the logistic regression models since the SVM model relies on the few support vectors. </p>

### Model 3: KNN

In [ ]:
# Create data frame to load in the accuracy score for each K chosen
points = pd.DataFrame(columns=['k_score'])

# Run thorugh different K neighbors and load into a dataframe
for K in range(1,30):
    clf_knn = KNeighborsClassifier(n_neighbors=K, weights='uniform', metric='euclidean')
    clf_knn.fit(x_train,y_train)
    preds = clf_knn.predict(x_test)
    
    points.loc[K] = [accuracy_score(y_test,preds)]
    # print('KNN Accuracy of classifier with %d neighbors is: %.2f'%(K,acc))

In [ ]:
# print(points)

plt.plot(points.k_score)
plt.title("Accuracy for K Neighbors")
plt.xlabel('K Neighbors')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
clf_knn = KNeighborsClassifier(n_neighbors=6, weights='uniform', metric='euclidean')
clf_knn.fit(x_train,y_train)
preds = clf_knn.predict(x_test)
    
acc = accuracy_score(y_test,preds)
print(acc)

### Explanation 

K Nearest Neighbors classification was run using a range of k values from 1-30. The accuracy within each model were visualized by the number chosen for k. From this graph, we were able to determine that a model that classifies based on 6 nearest neighbors would provide the highest accuracy. We created a model using knn with 6 as the k value for nearest neighbors. This model resulted in an accuracy score of 83.82%. 

### Model 4: Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Model 
forest = RandomForestClassifier()
# Fitting the Model
forest.fit(x_train,y_train)
# Predicting the Model
preds = forest.predict(x_test)
# Obtaining and printing the accuracy
f = accuracy_score(y_test,preds)
print("Random Forest Accuracy: ",f)


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# Building a confusion matrix
cm = confusion_matrix(y_test,preds)

print(cm)

from seaborn import heatmap
heatmap(cm/np.sum(cm), annot=True, fmt='.2%', cmap='Reds')
plt.title('Random Forest Prediction Accuracy Heat Map')

print(classification_report(y_test,preds))


In [ ]:
# Model 
forestTwo = RandomForestClassifier(max_depth=4)
# Fitting the Model
forestTwo.fit(x_train,y_train)
# Predicting the Model
predsTwo = forestTwo.predict(x_test)
# Obtaining and printing the accuracy
g = accuracy_score(y_test,predsTwo)
print("Random Forest Accuracy: ",g)

In [ ]:
cmTwo = confusion_matrix(y_test,predsTwo)

print(cmTwo)
heatmap(cmTwo/np.sum(cmTwo), annot=True, fmt='.2%', cmap='Reds')
plt.title('Random Forest Prediction Accuracy Heat Map')

print(classification_report(y_test,predsTwo))

### Explanation 

A random forest model was run without any controls on how the classification would be constructed. This initial model resulted in an accuracy of 83.60%. An alternative random forest model was run, this time limiting the size of the tree to a max depth of 4. This alternative random forest model resulted in an accuracy of 84.5%. Through the visualizations of the heatmaps, we can see where the tradeoff lies. For instance, limiting the tree to have a max depth of 4 nodes, causes over a 2% loss of proportion of individuals who were correctly classified as having an income greater than 50k. 

## Modeling and Evaluation: Part 4
*Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that that might use this model.*

--- INSERT EXPLANATION --- 

## Modeling and Evaluation: Part 5
*Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical methods.*

--- INSERT EXPLANATION --- 

## Modeling and Evaluation: Part 6
*Which attributes from your analysis are more important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classification task. *

--- INSERT EXPLANATION --- 

## Deployment

*How useful is your model for interested parties (i.e. the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would you deploy your mocel for interested parties? What other data should be collected? How often would the model need to be updated, etc?*

--- INSERT ANSWER --- 


## Exceptional Work

*One idea: Grid search parameters in a parallelized fashion and visualize the performances across attributes. Which parameters are most significant for making a good model for each classification algorithm? 

In [ ]:
# fit all models
clf = LazyClassifier(predictions=True)
models, predictions = clf.fit(x_train, x_test, y_train, y_test)